In [1]:
import sys
sys.path.append('/Users/katiehuang/Documents/metis/projects/onl_ds5_project_2/py')
from loadpage import *
import importlib
from fictiondb import *

In [1]:
from bs4 import BeautifulSoup
import requests
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [2]:
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [3]:
# # Open automated Chrome with fictionDB webpage
driver = webdriver.Chrome(chromedriver)
driver.get("https://www.fictiondb.com/search/search.htm")

In [8]:
def search(author):
    author_dict={}

    driver.get("https://www.fictiondb.com/search/search.htm")
    search_bar = driver.find_element_by_xpath("//input[@name='author'][@type='text']")
    search_bar.clear()
    search_bar.send_keys(author)
    search_bar.send_keys(Keys.RETURN)
    
    # Click on the author link
    try:

        author_r = ", ".join(author.split()[::-1])
        elem = driver.find_element_by_link_text(author_r)
        elem.click()
    except:
        print("can't find author")
        pass

    try:
        publication = driver.find_elements_by_xpath("//a[@itemprop='datePublished']")
        dates = [s.text.split('-') for s in publication]
        years = [int(y[-1]) for y in dates]
    except:
        print("can't find publication")
        pass
    
    author_dict[author] = years

    return author_dict

In [9]:
search('Jane Austen')

{'Jane Austen': [1818,
  1817,
  1815,
  1814,
  1813,
  1811,
  1805,
  1805,
  1803,
  1790,
  1786]}

In [3]:
di={}
di["jane"] = [1,2,3]

In [4]:
di

{'jane': [1, 2, 3]}

In [3]:
try:
    publication = driver.find_elements_by_xpath("//a[@itemprop='datePublished']")
    dates = [s.text.split('-') for s in publication]
    years = [int(y[-1]) for y in dates]
except:
    print("can't find publication")
    pass

In [4]:
years

[1818, 1817, 1815, 1814, 1813, 1811, 1805, 1805, 1803, 1790, 1786]

In [6]:
di={}
author="Jane"
di[author] = years

In [7]:
di

{'Jane': [1818, 1817, 1815, 1814, 1813, 1811, 1805, 1805, 1803, 1790, 1786]}